In [2]:
import pandas as pd

In [3]:
rankings = pd.read_csv('Prelim2019_MasseyOrdinals/Prelim2019_MasseyOrdinals.csv')
seasons = pd.read_csv('Stage2DataFiles/Seasons.csv',parse_dates=['DayZero'],index_col='Season')

In [4]:
rankings['Date'] = rankings.apply(lambda row: pd.to_timedelta(row['RankingDayNum'],unit='D') + \
                                  seasons.loc[row['Season'],'DayZero'],axis=1) 

In [5]:
## save this in a dictionary with team ID as key
eff_rank = {t:rankings[rankings['TeamID'] == t].set_index('Date').resample('7D').mean().interpolate().\
            rolling(4).mean().bfill()['OrdinalRank'] \
               for t in rankings['TeamID'].unique()}

In [6]:
df = pd.concat([pd.read_csv('Stage2DataFiles/RegularSeasonCompactResults.csv'),
               pd.read_csv('Stage2DataFiles/NCAATourneyCompactResults.csv')]).reset_index(drop=True)
df = df[df['Season'] >= 2003].reset_index(drop=True)
df = df.sort_values(by=['Season','DayNum'])

In [7]:
df['Date'] = df.apply(lambda row: pd.to_timedelta(row['DayNum'],unit='D') \
                          + seasons.loc[row['Season'],'DayZero'],axis=1)

In [8]:
for i,game in df.iterrows():
    Wteam_rank = eff_rank[game['WTeamID']].reset_index()
    Widx = (Wteam_rank['Date'] - game['Date']).abs().idxmin()
    df.at[i,'Wrank'] = Wteam_rank.loc[Widx,'OrdinalRank']
    Lteam_rank = eff_rank[game['LTeamID']].reset_index()
    Lidx = (Lteam_rank['Date'] - game['Date']).abs().idxmin()
    df.at[i,'Lrank'] = Lteam_rank.loc[Lidx,'OrdinalRank']

In [9]:
df.to_csv('rankings.csv')

In [10]:
df.head()

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,Date,Wrank,Lrank
0,2003,10,1104,68,1328,62,N,0,2002-11-14,17.364450,29.844785
1,2003,10,1272,70,1393,63,N,0,2002-11-14,57.376375,54.096675
2,2003,11,1266,73,1437,61,N,0,2002-11-15,22.799617,61.893478
3,2003,11,1296,56,1457,50,N,0,2002-11-15,183.660838,229.757481
4,2003,11,1400,77,1208,71,N,0,2002-11-15,14.832990,46.966049
